In [2]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, GlobalAveragePooling1D, Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
from google.colab import drive
import os

# 1. Mount Drive
drive.mount('/content/drive')
BASE_DIR = "/content/drive/MyDrive/PTBXL"

# 2. Load Data
train_file = f"{BASE_DIR}/ptbxl_train_preprocessed_100hz_10s.npz"
data = np.load(train_file, allow_pickle=True)
X_train = data['X']
y_train = data['y']
class_names = data['class_names'] if 'class_names' in data else [str(i) for i in np.unique(y_train)]

print(f"🔥 Training Pro Model on {X_train.shape[0]} samples...")

# 3. Build the "Pro" Model (More robust against overfitting)
model = Sequential([
    # Layer 1
    Conv1D(64, kernel_size=10, activation='relu', input_shape=(1000, 12)),
    BatchNormalization(),
    MaxPooling1D(pool_size=2),

    # Layer 2
    Conv1D(128, kernel_size=10, activation='relu'),
    BatchNormalization(),
    MaxPooling1D(pool_size=2),
    Dropout(0.4),  # Increased Dropout

    # Layer 3
    Conv1D(256, kernel_size=10, activation='relu'),
    BatchNormalization(),
    MaxPooling1D(pool_size=2),
    Dropout(0.5),  # Maximum Dropout

    # Output
    GlobalAveragePooling1D(),
    Dense(128, activation='relu'),
    Dropout(0.5), # Extra safety layer
    Dense(len(class_names), activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# 4. Define Smart Callbacks (The Secret Sauce)
callbacks = [
    # Save the BEST model, not the last one
    ModelCheckpoint(
        filepath=f"{BASE_DIR}/final_heart_model.keras",
        save_best_only=True,
        monitor='val_accuracy',
        mode='max',
        verbose=1
    ),
    # Slow down learning if stuck
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, verbose=1),
    # Stop early if not improving (saves time)
    EarlyStopping(monitor='val_loss', patience=7, restore_best_weights=True)
]

# 5. Train (Longer, but smarter)
history = model.fit(
    X_train, y_train,
    epochs=30,  # Give it more time to learn slowly
    batch_size=32,
    validation_split=0.2,
    callbacks=callbacks
)

print(f"🎉 SUCCESS! The BEST version of your model was saved to {BASE_DIR}/final_heart_model.keras")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
🔥 Training Pro Model on 17166 samples...
Epoch 1/30
430/430 ━━━━━━━━━━━━━━━━━━━━ 0s 597ms/step - accuracy: 0.5049 - loss: 2.0586
Epoch 1: val_accuracy improved from -inf to 0.56232, saving model to /content/drive/MyDrive/PTBXL/final_heart_model.keras
430/430 ━━━━━━━━━━━━━━━━━━━━ 281s 646ms/step - accuracy: 0.5050 - loss: 2.0580 - val_accuracy: 0.5623 - val_loss: 1.6508 - learning_rate: 0.0010
Epoch 2/30
430/430 ━━━━━━━━━━━━━━━━━━━━ 0s 623ms/step - accuracy: 0.6133 - loss: 1.4992
Epoch 2: val_accuracy improved from 0.56232 to 0.58561, saving model to /content/drive/MyDrive/PTBXL/final_heart_model.keras
430/430 ━━━━━━━━━━━━━━━━━━━━ 333s 672ms/step - accuracy: 0.6133 - loss: 1.4992 - val_accuracy: 0.5856 - val_loss: 1.4659 - learning_rate: 0.0010
Epoch 3/30
430/430 ━━━━━━━━━━━━━━━━━━━━ 0s 767ms/step - accuracy: 0.6237 - loss: 1.4034
Epoch 3: val_accuracy improve